# Colab output wrapper

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# wrap the output in colab cells
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

# Install Transformers

In [ ]:
# install transformers with sentencepiece
!pip install transformers[sentencepiece]

# Read input file from Google Drive

In [ ]:
# open and read the file from google drive
file = open("/content/drive/MyDrive/to be delete/judgment/78.txt", "r")
FileContent = file.read().strip()

In [ ]:
# display file content
FileContent

In [ ]:
# total characters in the file
len(FileContent)

71691

# Load the Model and Tokenizer

In [ ]:
# import and initialize the tokenizer and model from the checkpoint
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

checkpoint = "sshleifer/distilbart-cnn-12-6"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

# Some model statistics

In [ ]:
# max tokens including the special tokens
tokenizer.model_max_length

1024

In [ ]:
# max tokens excluding the special tokens
tokenizer.max_len_single_sentence

1022

In [ ]:
# number of special tokens
tokenizer.num_special_tokens_to_add()

2

# Convert file content to sentences

In [ ]:
# extract the sentences from the document
import nltk
nltk.download('punkt')
sentences = nltk.tokenize.sent_tokenize(FileContent)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# find the max tokens in the longest sentence
max([len(tokenizer.tokenize(sentence)) for sentence in sentences])

137

# Create the chunks

In [ ]:
# initialize
length = 0
chunk = ""
chunks = []
count = -1
for sentence in sentences:
  count += 1
  combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

  if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed
    chunk += sentence + " " # add the sentence to the chunk
    length = combined_length # update the length counter

    # if it is the last sentence
    if count == len(sentences) - 1:
      chunks.append(chunk.strip()) # save the chunk

  else:
    chunks.append(chunk.strip()) # save the chunk

    # reset
    length = 0
    chunk = ""

    # take care of the overflow sentence
    chunk += sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(chunks)



3

In [ ]:
chunk_list=[]
chunk_list=chunks

# Some checks

In [ ]:
[len(tokenizer.tokenize(c)) for c in chunks]

[1006, 1008, 827]

In [ ]:
[len(tokenizer(c).input_ids) for c in chunks]

[1008, 1010, 829]

## With special tokens added

In [ ]:
sum([len(tokenizer(c).input_ids) for c in chunks])

2847

In [ ]:
len(tokenizer(FileContent).input_ids)

2909

## Without special tokens added

In [ ]:
sum([len(tokenizer.tokenize(c)) for c in chunks])

2841

In [ ]:
len(tokenizer.tokenize(FileContent))

2907

# Get the inputs

In [ ]:
# inputs to the model
inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks]

# Output

In [ ]:
summary_list=[]
for input in inputs:
  output = model.generate(**input)
  summary_list.append(tokenizer.decode(*output, skip_special_tokens=True))
print(summary_list)

[' Appeal from an Order of the High Court of Bombay (Bavdekar and Chainani JJ.) dated 20th February, 1950, in Criminal Appeal No. 106 of 1950. W.H. King was tenant of a flat on the second floor of a building called "Ganga Vihar", Marine Drive, Bombay, which belongs to a lady named Durgeshwari Devi.', ' The wife of the appellant denied any aiding and abetting. The wife was acquitted, the evidence being insufficient to prove any abetment. The appellant preferred an appeal to the High Court of Bombay but it was summarily dismissed. He asked for a certificate under article 134(1)(c) of the Constitution but this was rejected on 10 4 1950. He applied for special leave to appeal to this Court and it was granted on 3 10 1950.', ' The word "relinquishment" does not occur in the Transfer of Property Act but it is found in many of the Tenancy Acts in various provinces where there are Sec tions which deal with the relinquishment of their holdings by tenants in favour of the landlord by notice give

Convert list to dataframe

In [ ]:
# Convert lists to DataFrame
#data = pd.DataFrame({"Chunk": chunk_list, "Summary": summary_list})
# Write DataFrame to Excel file
#data.to_excel("/content/drive/MyDrive/to be delete/getreadings.xlsx", index=False)
import os
import pandas as pd
# If the file exists, load existing data and append new data
# Check if the file exists
file_path = "/content/drive/MyDrive/to be delete/test_data.xlsx"
file_exists = os.path.isfile(file_path)

if file_exists:
    existing_data = pd.read_excel(file_path)
    combined_data = pd.concat([existing_data, pd.DataFrame({"Chunk": chunk_list, "Summary": summary_list})], ignore_index=True)
else:
    combined_data = pd.DataFrame({"Chunk": chunk_list, "Summary": summary_list})

# Write combined data to Excel file
combined_data.to_excel(file_path, index=False)

evaluation[link text](https://)

# Blanc Evaluation

In [ ]:
# extract the sentences from the document
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
pip install blanc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 875.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 62.7 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.2.1+cu121
    Uninstalling torch-2.2.1+cu121:
      Successfully uninstalled torch-2.2.1+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 1.13.1 which is incompatible.
torchdata 0.7.1 requires torch>=2, but you have torch 1.13.1 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 1.13.1 wh

In [ ]:
document = """
N: Criminal Appeal No. 8 of 1951.
Appeal from an Order of the High Court of Bombay (Bavdekar and Chainani JJ.) dated 20th February, 1950, in Criminal Appeal No. 106 of 1950 arising out of an order dated 9th January, 1950, of the Presidency Magistrate, 19th Court, Esplanade, Bombay, in Case No. 10879/P of 1949.
The facts are stated in the judgment.
Iswarlal C. Dalai and R.B. Dalai, for the appellant.
C.K. Daphtary, Solicitor General for India (G. N. Joshi, with him) for the Republic of India (respondent No. 1).Jindra Lal for the respondent No. 2. 1952.
February 1.
The Judgment of the Court was deliv ered by CHANDRASEKHARA AIYAR J.
The facts out of which this Crimi nal Appeal has arisen are not long.
The appellant, W.H. King, who is carrying on a business in Bombay under the name and style of Associated Commercial Enterprises, was the tenant of a flat on the second floor of a building called "Ganga Vihar", Marine Drive, Bombay, which belongs to a lady named Durgeshwari Devi.
The tenancy was a monthly one, the rent being Rs. 215.
It is said that the appellant wanted to go to the United Kingdom for treatment of his failing eye sight and he got into touch with the complainant Mulchand Kodumal Bhatia, who is the second respondent in this appeal, through one Sayed for the purpose of making necessary ar rangements about the flat occupied by him in view of his intended departure.
The prosecution case is that the accused demanded a sum of Rs. 30,000 which was later on reduced to Rs. 29,500 as consideration for putting the complainant in vacant possession of the flat and an additional amount of Rs. 2,000 for the furniture, and that the complainant agreed to pay these sums.
The complainant actually paid the accused two sums of 420 Rs. 500 each on 7th November, 1948, and 17th November, 1948.
He, however, got into touch with the police on 1 12 1948, and in conjunction with the latter, a trap was laid for the appellant.
It was arranged that the complainant should bring with him Rs. 1,000, being the balance due in respect of the furniture and that the police would give him Rs. 29,500 to be paid to the appellant.
The complainant and a Sub Inspec tor, posing as the complainant 's brother, went to the appel lant on 4 12 1948, and paid him the two sums of money; and the keys of the flat and the motor garage were handed over to the complainant.
As the appellant and his wife were leaving the flat, the man, who masqueraded as the complain ant 's brother, threw off his disguise and disclosed his identity.
The police party, who were down below ready for the raid, held up the car of the appellant and recovered the sum of Rs. 30,500 from the rear seat of the car and also some papers, a typed draft of a partnership agreement be tween the complainant and the appellant and an application form for permission to occupy the building as caretaker.
From the complainant were recovered the bunch of keys and the documents that were handed over to him by the appellant, namely, the letter handing vacant possession (Exhibit D).
the receipt for Rs. 2,000 for the articles of furniture (Exhibit E), a letter to the Bombay Gas Company for transfer of the gas connection to the name of the complainant (Exhib it F), and the letter to the Bombay Electric Supply and Transport Committee for transfer of the telephone connec tions and the deposit of Rs. 27 (Exhibit G).
The appellant was charged under section 18(1) of the Bombay Rents, Hotel and Lodging House Rates Control Act, LVII of 1947, for receiving a pugree of Rs. 29,500 and he was further charged under section 19(2) of the said Act for receiving the said sum as a condition for the relin quishment of his tenancy.
His wife, who was the second accused in the case, was charged with aiding and abetting her husband in the commission of the two offences.
421 The defence of the appellant was that he was in search of a partner to carry on his business during his intended absence, who was also to act as caretaker of his flat anal that it was in this connection and with this object in view that he entered into negotiations with the complain ant.
The sum of Rs. 29 500 was not pugree but represented capital for 0 12 0 share in the business and as the com plainant was also to be a caretaker of the flat, the sum of Rs. 2,000 was paid and received as a guarantee against disposal and damage of the furniture and it was agreed to be paid back on the appellant 's return to India.
The wife of the appellant denied any aiding and abetting.
The Presidency Magistrate, who tried the case, disbe lieved the defence on the facts, holding that what was received by the accused was by way of pugree.
As section 18 (1) of the Act was not applicable he convicted him under section 19(2) of the Act and sentenced him, in view of his old age and blindness, to one day 's simple imprisonment and a fine of Rs. 30,000.
The wife was acquitted, the evidence being insufficient to prove any abetment.
The appellant preferred an appeal to the High Court of Bombay but it was summarily dismissed on 20 2 1950.
He asked for a certificate under article 134(1)(c) of the Constitution but this was rejected on 10 4 1950.
Thereaf ter he applied for special leave to appeal to this Court and it was granted on 3 10 1950.
A short legal argument was advanced on behalf of the appellant based on the language of section 19 (1) of the Act and this is the only point which requires our consideration.
The section which consists of two parts is in these terms:" "(1) It shall not be lawful for the tenant or any person acting or purporting to act on behalf of the tenant to claim or receive any sum or any consideration as a condi tion for the relinquishment of his tenancy of any premises; 422 (2) Any tenant or person who in contravention of the provisions of sub section (1) receives any sum or considera tion shall, on conviction, be punished with imprisonment for a term which may extend to 6 months and shall also be pun ished with fine which shall not be less than the sum or the value of the consideration received by him.
" It was urged that the offence arises only on receipt of any sum or any consideration as a condition of the relin quishment by a tenant of his tenancy and that in the present case there was no such relinquishment.
Exhibit D, which is the most material document, under which the appellant handed over vacant possession of the flat to the complainant, constitutes or evidences an assignment of the tenancy and not a relinquishment.
It says : "I, W.H. King, hereby hand over vacant possession of my flat No. 3 situated on 2nd floor and garage No. 4 on the ground floor of Ganga Vihar Building on Plot No. 55 situated on Marine Drive Road to Mr. Mulchand Kodumal Bhatia from this day onward and that I have no claim whatsoever over this flat and Mr. Mulchand Kodumal Bhatia will pay the rent directly to the landlord.
" The argument raised on behalf of the appellant appears to us to be sound and has to be accepted.
The learned Solic itor General urged that 'the word "relinquishment" was not a term of art and was used in the section not in any strict technical sense but in its comprehensive meaning as giving up of possession of the premises; and he pointed out that if it was intended by the legislature that "relinquish ment" should have the limited meaning sought to be placed upon it on behalf of the appellant, the word "surrender" used in the Transfer of Property Act would have been more appropriate.
Sections 15 and 18 of the Act were referred to in this connection but in our opinion they lend no assist ance to the argument of the learned counsel.
Any sublet ting, assignment or transfer in any other manner of his interest by the tenant is made unlawful under 423 section 15.
Section 18 deals with the grant, renewal or continuance of a lease of any premises or the giving of his consent by the landlord to the transfer of a lease by sub lease or otherwise, and it provides that the landlord, who receives any fine, premium, or other like sum or deposit, or any consideration for the grant, renewal or continuance or the accord of consent oh would be guilty of an offence and liable to the punishment therein specified.
It would thus be seen that an assignment of the lease or transfer in any other manner by a tenant is not made an offence; the statute merely says that it is not a lawful transaction.
It is the landlord 's consent to the transfer of a lease by sub lease or otherwise on receipt of consideration that has been made an offence.
Then follows section 19 which speaks of the relinquishment of his tenancy of any premises by a tenant.
If, by the expression, an assignment such as we have in the present case was meant, appropriate words could have been used, such as the transfer by a tenant of his interest, which we find in section 108, sub clause (i), of the Trans fer of Property Act.
The distinction between an assignment on the one hand and relinquishment or surrender on the other is too plain to be ignored.
In the case of an assignment, the assignor contin ues to be liable to the landlord for the performance of his obligations under the tenancy and this liability is contrac tual, while the assignee becomes liable by reason of privity of estate.
The consent of the landlord to an assignment is not necessary, in the absence of a contract or local usage to the contrary.
But in the case of relinquishment, it cannot be a unilateral transaction; it can only be in favour of the lessor by mutual agreement between them.
The relin quishment of possession must be to the lessor or one who holds his interest.
In fact, a surrender or relinquishment terminates the lessee 's rights and lets in the lessor.
It is no doubt true that the word "relinquishment" does not occur in the Transfer of Property Act but it is found in many of the Tenancy Acts in various provinces where there are Sec tions which deal with the 55 424 relinquishment of their holdings by tenants in favour of the landlord by notice given to him in writing.
The section in question, it should be further noted, does not speak of relinquishment or giving up of possession,in general terms.
The words are "the relinquishment of his tenancy of any premises".
The relinquishment of a tenancy is equivalent to surrender by the lessee or tenant of his rights as such.
Whether abandonment of a tenancy would come within the meaning of relinquishment is a question that does not arise in this appeal, because in the face of Exhibit D, there is no abandonment in the sense that the tenant disappeared from the scene altogether saying nothing and making no arrange ments about his interest and possession under the lease.
As the statute creates an offence and imposes a penalty of fine and imprisonment, the words of the section must be strictly construed in favour of the subject.
We are not concerned so much with what might possibly have been intend ed as with what has been actually said in and by the language employed.
As in our view, there has been no "relinquishment" within the meaning of section 19, sub clause (1), the conviction under sub clause (2) cannot be sustained.
It is set aside and the fine of Rs. 30,000 will be refunded if it has al ready been paid.
The other parts of the order of the learned Presidency Magistrate, as regards the disposal of Rs. 1,000 paid by the complainant to the appellant and the sum of Rs. 29,500 brought in by the police, will, however, stand.
Conviction sit aside.
Agent for respondent No. 1: P.A. Mehta.
Agent for respondent No. 2: Ganpat Rai.

"""


summary = """

 Appeal from an Order of the High Court of Bombay (Bavdekar and Chainani JJ.) dated 20th February, 1950, in Criminal Appeal No. 106 of 1950. W.H. King was tenant of a flat on the second floor of a building called "Ganga Vihar", Marine Drive, Bombay, which belongs to a lady named Durgeshwari Devi.', ' The wife of the appellant denied any aiding and abetting. The wife was acquitted, the evidence being insufficient to prove any abetment. The appellant preferred an appeal to the High Court of Bombay but it was summarily dismissed. He asked for a certificate under article 134(1)(c) of the Constitution but this was rejected on 10 4 1950. He applied for special leave to appeal to this Court and it was granted on 3 10 1950.', ' The word "relinquishment" does not occur in the Transfer of Property Act but it is found in many of the Tenancy Acts in various provinces where there are Sec tions which deal with the relinquishment of their holdings by tenants in favour of the landlord by notice given to him in writing. The conviction under sub clause (2) cannot be sustained, the conviction is set aside and the fine of Rs. 30,000 will be refunded if it has been paid.']


"""

In [ ]:
from blanc import BlancHelp, BlancTune
blanc_help = BlancHelp()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
blanc_tune = BlancTune(finetune_mask_evenly=False, show_progress_bar=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
blanc_help.eval_once(document, summary)

100%|██████████| 336/336 [08:09<00:00,  1.46s/it]


0.11402623612512613

In [ ]:
blanc_tune.eval_once(document, summary)

100%|██████████| 168/168 [00:36<00:00,  4.63it/s]
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW inst

0.06458123107971746

# ROUGE Evaluation

In [ ]:
pip install rouge

In [ ]:
pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=03b90a86ea8782438c46f07f88b3224959401a2a94a114b4eb3cf5d8453cbfda
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
generated_summary="""

## Summary of Civil Misc. Petition No. 15001 of 1989

This case involved a property dispute between Subbamma, her adopted son Seshiah, and his two sons, Sudarshan Gupta and Anand Babu.

**Background:**

* Subbamma had two wills, each leaving the property to a different son.
* Seshiah claimed the property as Subbamma's heir.

**Compromise Attempt:**

* During the legal proceedings, a compromise was proposed.
* Seshiah agreed to pay each son Rs. 1 lakh in exchange for relinquishing their claims.
* The compromise document was signed by all parties.

**Dispute over Compromise:**

* Sudarshan Gupta (second son) later denied receiving the Rs. 1 lakh and refused the compromise.
* He also sold parts of the property to third parties (alienees).

**Court's Decision:**

* The Court considered the compromise document valid despite Sudarshan Gupta's objection.
* The Court found the property sales to be invalid as they occurred after the compromise.
* The alienees (who bought the property) were ordered to:
    * Deposit the sale consideration they paid (Rs. 77,124) in court.
    * Vacate the property by April 30th, 1991.

**Additional Orders:**

* Seshiah was ordered to pay Sudarshan Gupta an additional Rs. 1 lakh.
* The alienees were relieved of any obligation to pay rent for occupying the property.

**Outcome:**

* The compromise was upheld.
* Seshiah regained ownership of the property.
* Sudarshan Gupta received a total of Rs. 2 lakh.
* The alienees lost their claims on the property and any money they paid towards the purchase.

"""

reference_summary="""
Respondent No. 1 adopted petitioner appellant, who married two wives and through the first wife he had a son, the respondent No. 2 and through the second, another son, the respondent No. 3.
During the pendency of the special leave petition the adoptive mother of appellant died.
Respondent 2 and 3 laid claim to the entire property of respondent No. 1 exclusively under two different wills said to be by the respondent No. 1 and each contended that the other will was a forged one.
On the death of the adoptive mother, the appellant laid claim to her entire property as heir.
While each of the parties had taken such stand, in the litigation a compromise was brought about on 21.8.1987 between the appellant and his two sons, the respondents 2 and 3 and the same was filed in this Court and in terms of the compromise the appellant, to make payment of Rs. 1 lakh to each of his two sons in lieu of relinquishment of their interest.
When the matter was listed for recording the compromise, the respondent No. 2 contended that as he had not been paid Rs. 1 lakh as stipulated in the compromise, in the meanwhile he had alienated about 81 acres of the suit properties to the third parties.
The alienees had been also impleaded as parties under the orders of this Court.
Disposing of the petition, this Court, HELD: 1.
Once the Court was satisfied that there was a compromise it was for the Court to record the same and no option lay before the Court to act otherwise.
[542A B] 539 2.
As the compromise petition in the instant case is genuine nd lawful the same has to be acted upon.
[542B] 3.
It is directed that the compromise petition shall be accepted and in terms thereof the suit shall be disposed of and the terms of the compromise shall form part of the order to be drawn up in this Court for disposing of the special leave petition.
[542B C] 4.
With a view to settling all equities between the parties, directed that the appellant to pay a sum of Rs.1 lakh more to respondent No. 2 within eight weeks.
Rs. 77,124 being the amount paid by the alienees before the Sub Registrar in respect of the sale deeds shall be deposited.
[542C E] 5.
The alienees have no right created under the alleged sale deeds.
Their possession is without authority of law and clarified that none of the sale deeds is valid.
[542F H] 6.
The alienees shall deliver vacant possession of the property by 30th of April, 1991, and in the event of failure to do so the Trial Court directed to deliver vacant possession.
[543A B] Bhoja Govinda Maikap & Anr.
vs Janaki Dei & Ors., , approved.

"""

In [ ]:
from rouge import Rouge
def calculate_rouge_scores(generated_summary, reference_summary):
    rouge = Rouge()
    scores = rouge.get_scores(generated_summary, reference_summary)
    return scores

# Define function to calculate accuracy
def calculate_accuracy(generated_summary, reference_summary):
    # Tokenize the summaries
    generated_tokens = nltk.word_tokenize(generated_summary.lower())
    reference_tokens = nltk.word_tokenize(reference_summary.lower())

    # Calculate accuracy based on matching tokens
    matching_tokens = set(generated_tokens) & set(reference_tokens)
    accuracy = len(matching_tokens) / len(reference_tokens)
    return accuracy

In [ ]:
# Calculate ROUGE scores
rouge_scores = calculate_rouge_scores(generated_summary, reference_summary)
print("ROUGE scores:", rouge_scores)

# Calculate accuracy
accuracy = calculate_accuracy(generated_summary, reference_summary)
print("Accuracy:", accuracy)

ROUGE scores: [{'rouge-1': {'r': 0.2, 'p': 0.31654676258992803, 'f': 0.24512534344395226}, 'rouge-2': {'r': 0.049738219895287955, 'p': 0.09090909090909091, 'f': 0.06429779576684708}, 'rouge-l': {'r': 0.18636363636363637, 'p': 0.2949640287769784, 'f': 0.22841225152194672}}]
Accuracy: 0.10857142857142857


# Saving the DistillBART summary output in the File

In [ ]:
pip install openpyxl

In [ ]:
import os
import openpyxl  # For writing to Excel
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk

# Define directory path
judgment_dir = "/content/drive/MyDrive/to be delete/judgment"  # Modify as needed

# Pre-trained model checkpoint (replace with your desired model)
checkpoint = "sshleifer/distilbart-cnn-12-6"


def summarize_text(file_content):
  """
  This function takes the text content of a file and returns a list of summaries for chunks.
  """
  tokenizer = AutoTokenizer.from_pretrained(checkpoint)
  model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

  nltk.download('punkt')
  sentences = nltk.tokenize.sent_tokenize(file_content)

  length = 0
  chunk = ""
  chunks = []
  count = -1
  for sentence in sentences:
    count += 1
    combined_length = len(tokenizer.tokenize(sentence)) + length
    if combined_length <= tokenizer.max_len_single_sentence:
      chunk += sentence + " "
      length = combined_length
    else:
      chunks.append(chunk.strip())
      length = 0
      chunk = sentence + " "
    if count == len(sentences) - 1:
      chunks.append(chunk.strip())

  chunk_list = chunks

  inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks]
  summary_list = []
  for input in inputs:
    output = model.generate(**input)
    summary_list.append(tokenizer.decode(*output, skip_special_tokens=True))

  return summary_list


def main():
  # Initialize Excel workbook and sheet
  wb = openpyxl.Workbook()
  sheet = wb.active
  sheet.cell(row=1, column=1).value = "Filename"
  sheet.cell(row=1, column=2).value = "Summary"

  # Iterate through files in the directory
  for filename in os.listdir(judgment_dir):
    if filename.endswith(".txt"):  # Filter for text files
      filepath = os.path.join(judgment_dir, filename)
      try:
        # Read file, summarize, and save summary
        with open(filepath, "r") as file:
          file_content = file.read().strip()
        summary_list = summarize_text(file_content)

        # Write filename and summary to Excel
        row = sheet.max_row + 1
        sheet.cell(row=row, column=1).value = filename
        for i, summary in enumerate(summary_list):
          sheet.cell(row=row + i, column=2).value = summary

      except Exception as e:  # Handle potential errors
        print(f"Error processing file {filename}: {e}")

  # Save the Excel workbook
  wb.save("judgment_summaries.xlsx")  # Modify filename as needed
  print("Summaries saved to judgment_summaries.xlsx")


if __name__ == "__main__":
  main()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_

Summaries saved to judgment_summaries.xlsx
